# Прямая задача кинематики PUMA

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from IPython.display import HTML
%matplotlib notebook

In [ ]:
from kinematics import Vector, Quaternion, Transform
import graphics

## Решение прямой задачи кинематики для манипулятора PUMA

Манипулятор кинематической схемы PUMA обладает шестью степенями подвижности.

![внешний виж PUMA](fig/puma_model.png)

Его можно условно разделить на сегменты, соединияюще между собой:
- основание
- плечо
- локоть
- кисть
- фланец

![кинематическая схема PUMA](fig/puma.png)

Длина этих сегментов определена в таблице:

Пара | длина
:----|:-----
основание - плечо | $l_0$
плечо - локоть | $l_1$
локоть - кисть | $l_2$
кисть - фланец | $l_3$

In [ ]:
def puma_chain(q, l, math_source=np):
    base = Transform.identity()
    shoulder = base + Transform(
        Vector(0, 0, l[0]),
        Quaternion.from_angle_axis(q[0], Vector(0, 0, 1), math_source) * 
        Quaternion.from_angle_axis(q[1], Vector(0, 1, 0), math_source)
    )
    elbow = shoulder + Transform(
        Vector(0, 0, l[1]),
        Quaternion.from_angle_axis(q[2], Vector(0, 1, 0), math_source)
    )
    wrist = elbow + Transform(
        Vector(0, 0, l[2]),
        Quaternion.from_angle_axis(q[3], Vector(0, 0, 1), math_source) *
        Quaternion.from_angle_axis(q[4], Vector(0, 1, 0), math_source)
    )
    flange = wrist + Transform(
        Vector(0, 0, l[3]),
        Quaternion.from_angle_axis(q[5], Vector(0, 0, 1), math_source)
    )
    return [base, shoulder, elbow, wrist, flange]

Зададим изменение обобщенных координат:

In [ ]:
def puma_q(t, total):
    omega = t / total * np.pi * 2
    return [
        np.pi / 4 * np.sin(omega),
        np.pi / 8,
        np.pi / 2,
        omega,
        np.pi / 2,
        0
    ]

Зададим длины звеньев:

In [ ]:
puma_l = [1, 2, 1, 0.5]

In [ ]:
puma_fig = plt.figure()
ax = puma_fig.add_subplot(projection="3d")
ax.set_xlim([-2, 2]); ax.set_ylim([-2, 2]); ax.set_zlim([0, 4])
lines, = ax.plot([], [], [], color="#000000")
graphics.axis(ax, Transform.identity(), 2)
r, g, b = graphics.axis(ax, Transform.identity(), 1)

total = 100

def animate(frame):
    chain = puma_chain(puma_q(frame, total), puma_l)
    (x, y, z) = graphics.chain_to_points(chain)
    lines.set_data_3d(x, y, z)
    global r, g, b
    r.remove(); g.remove(); b.remove()
    r, g, b = graphics.axis(ax, chain[-1], 0.5)

    
animate(0)
fps = 25
puma_ani = animation.FuncAnimation(
    puma_fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
HTML(puma_ani.to_jshtml())

## Отображение траектории

In [ ]:
t_end = 10
step = 0.1
t = np.arange(0, t_end, step)
chain = puma_chain(puma_q(t, t_end), puma_l)
x, y, z = graphics.chain_to_points(chain)

fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.set_xlim([-2, 2]); ax.set_ylim([-2, 2]); ax.set_zlim([0, 4])
ax.plot(x[2], y[2], z[2], color="#c0c0c0")
ax.plot(x[3], y[3], z[3], color="#c0c0c0")
ax.plot(x[4], y[4], z[4], color="#000000")
graphics.axis(ax, Transform.identity(), 2)
fig.show()

## Измерение скорости движения

Численно продиффиринцируем положение всех точек траектории

In [ ]:
t_end = 10
step = 0.01
t = np.arange(0, t_end, step)
chain = puma_chain(puma_q(t, t_end), puma_l)
end = chain[-1]
velocity_x = np.diff(end.translation.x) / step
velocity_y = np.diff(end.translation.y) / step
velocity_z = np.diff(end.translation.z) / step
velocity_total = (velocity_x ** 2 + velocity_y ** 2 + velocity_z ** 2) ** 0.5

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(t[:-1], velocity_x, color="#ff0000", label="$V_x$")
ax.plot(t[:-1], velocity_y, color="#00ff00", label="$V_y$")
ax.plot(t[:-1], velocity_z, color="#0000ff", label="$V_z$")
ax.plot(t[:-1], velocity_total, color="#000000", label="$V$")
fig.legend()
fig.show()

## Самостоятельные задания

Для своего закона изменения обобщенных координат

- Оцените диапазон $X$, $Y$ и $Z$ координат для данной траектории;
- Оцените максимальную скорость по осям;
- Оцените абсолютную максимальную скорость.